In [30]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [31]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [32]:
version = "gpt2"
sequence = "The quick brown fox jumps over the lazy dog."
max_length = 20

# GPT2Tokenizer

In [33]:
tokenizer: GPT2Tokenizer = GPT2Tokenizer.from_pretrained(version)
tokenizer

GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

In [34]:
# 需要手动设置pad_token
tokenizer.pad_token = tokenizer.eos_token

## tokenizer([sequence])

In [35]:
inputs = tokenizer(
    [sequence] * 2,                     # 句子batch
    truncation = True,                  # 超出max_length截断处理
    padding = True,                     # 填充方式选择 [True, 'longest', 'max_length', 'do_not_pad']
    # max_length = max_length,          # 最长长度,不设置默认为模型最大长度
    add_special_tokens = True,          # text添加特殊key
    return_length = True,               # 返回有效长度
    return_overflowing_tokens = False,  # 返回所有的文本片段（由于文本比较长，默认情况下超过预设截断长度的token会被丢失。如果设置了return_overflowing_tokens=True则会返回所有的token片段）。
    return_tensors = "pt"               # 返回数据格式 np pt tf jax
).to(device, torch.float16)    # https://github.com/huggingface/transformers/issues/16359

print(inputs.keys())
print(inputs["input_ids"])
print(inputs["attention_mask"]) # 对应是否是文字
print(inputs["length"])         # 对应有效文字长度

dict_keys(['input_ids', 'length', 'attention_mask'])
tensor([[  464,  2068,  7586, 21831, 18045,   625,   262, 16931,  3290,    13],
        [  464,  2068,  7586, 21831, 18045,   625,   262, 16931,  3290,    13]],
       device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
tensor([10, 10], device='cuda:0')


In [36]:
print(inputs["input_ids"])

tensor([[  464,  2068,  7586, 21831, 18045,   625,   262, 16931,  3290,    13],
        [  464,  2068,  7586, 21831, 18045,   625,   262, 16931,  3290,    13]],
       device='cuda:0')


# GPT2LMHeadModel

The GPT2 Model transformer with a language modeling head on top (linear layer with weights tied to the input embeddings).

In [38]:
model: GPT2LMHeadModel = GPT2LMHeadModel.from_pretrained(version, torch_dtype=torch.float16).to(device)
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [39]:
model.eval()
with torch.inference_mode():
    outputs = model(
        input_ids = inputs["input_ids"],
        attention_mask = inputs["attention_mask"],
    )
outputs

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[ -36.2874,  -35.0114,  -38.0793,  ...,  -40.5163,  -41.3760,
           -34.9193],
         [ -62.3139,  -61.5645,  -66.4939,  ...,  -68.1286,  -68.3228,
           -63.5830],
         [ -66.3240,  -66.7452,  -72.1618,  ...,  -75.1955,  -73.4650,
           -68.1786],
         ...,
         [ -88.8787,  -87.6110,  -92.3262,  ...,  -95.8310,  -93.5163,
           -91.9581],
         [ -90.3912,  -92.3085,  -96.8618,  ..., -104.7200,  -98.8052,
           -93.8644],
         [-108.0002, -105.9623, -108.8509,  ..., -119.0374, -116.7625,
          -100.9662]],

        [[ -36.2874,  -35.0114,  -38.0793,  ...,  -40.5163,  -41.3760,
           -34.9193],
         [ -62.3139,  -61.5645,  -66.4939,  ...,  -68.1286,  -68.3228,
           -63.5830],
         [ -66.3240,  -66.7452,  -72.1618,  ...,  -75.1955,  -73.4650,
           -68.1786],
         ...,
         [ -88.8787,  -87.6110,  -92.3262,  ...,  -95.8310,  -93.5163,
          

In [40]:
outputs.logits.shape

torch.Size([2, 10, 50257])

In [41]:
for past_key in outputs.past_key_values:
    for past in past_key:
        print(past.shape)
    print("-" * 25)

torch.Size([2, 12, 10, 64])
torch.Size([2, 12, 10, 64])
-------------------------
torch.Size([2, 12, 10, 64])
torch.Size([2, 12, 10, 64])
-------------------------
torch.Size([2, 12, 10, 64])
torch.Size([2, 12, 10, 64])
-------------------------
torch.Size([2, 12, 10, 64])
torch.Size([2, 12, 10, 64])
-------------------------
torch.Size([2, 12, 10, 64])
torch.Size([2, 12, 10, 64])
-------------------------
torch.Size([2, 12, 10, 64])
torch.Size([2, 12, 10, 64])
-------------------------
torch.Size([2, 12, 10, 64])
torch.Size([2, 12, 10, 64])
-------------------------
torch.Size([2, 12, 10, 64])
torch.Size([2, 12, 10, 64])
-------------------------
torch.Size([2, 12, 10, 64])
torch.Size([2, 12, 10, 64])
-------------------------
torch.Size([2, 12, 10, 64])
torch.Size([2, 12, 10, 64])
-------------------------
torch.Size([2, 12, 10, 64])
torch.Size([2, 12, 10, 64])
-------------------------
torch.Size([2, 12, 10, 64])
torch.Size([2, 12, 10, 64])
-------------------------


In [42]:
outputs.hidden_states

In [43]:
outputs.attentions

In [44]:
outputs.cross_attentions